# Roll hull equation

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sympy as sp
from sympy.plotting import plot as plot
from sympy.plotting import plot3d as plot3d
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sp.init_printing()
from IPython.core.display import HTML

In [ ]:
import seaman.helpers
import seaman_symbol as ss
import roll_hull_equations as equations
import roll_hull_lambda_functions as lambda_functions
from bis_system import BisSystem

## Coordinate system
![coordinate_system](coordinate_system.png)

## Symbols

In [ ]:
from seaman_symbols import *

In [ ]:
HTML(ss.create_html_table(symbols=equations.roll_hull_equation.free_symbols))

## Roll equation

In [ ]:
equations.roll_hull_equation

In [ ]:
equations.roll_hull_equation_SI

### Plotting the foll hull moment equation

In [ ]:
df = pd.DataFrame()
df['v_w'] = np.linspace(-0.3,3,10)
df['u_w'] = 5.0
df['p'] = 0
df['r_w'] = 0.0
df['rho'] = 1025
df['L'] = 1.0
df['g'] = 9.81
df['disp'] = 23
df['K_ur'] = 1 
df['K_uur'] = 1 
df['K_uv'] = 1
df['K_uuv'] = 1 
df['K_up'] = 1 
df['K_p'] = 1
df['K_vav'] = 1 
df['K_rar'] = 1 
df['K_pap'] = 1

result = df.copy()
lambda_functions.K_h_function(**df)
result['mx'] = lambda_functions.K_h_function(**df)

result.plot(x = 'v_w',y = 'mx');

### Plotting with coefficients from a real seaman ship model

In [ ]:
import generate_input
shipdict = seaman.ShipDict.load('../../tests/test_ship.ship')

In [ ]:
df = pd.DataFrame()
df['v_w'] = np.linspace(-3,3,20)
df['rho'] = 1025.0
df['g'] = 9.81
df['u_w'] = 5.0
df['r_w'] = 0.0
df['p'] = 0

df_input = generate_input.add_shipdict_inputs(lambda_function=lambda_functions.K_h_function,
                                              shipdict = shipdict,
                                              df = df)
df_input

In [ ]:
result = df_input.copy()
result['mx'] = lambda_functions.K_h_function(**df_input)

In [ ]:
result.plot(x = 'v_w',y = 'mx');

## Real seaman++
Run real seaman in C++ to verify that the documented model is correct. 

In [ ]:
import run_real_seaman

In [ ]:
df = pd.DataFrame()
df['v_w'] = np.linspace(-3,3,20)
df['rho'] = 1025.0
df['g'] = 9.81
df['u_w'] = 5.0
df['r_w'] = 0.0
df['p'] = 0

result_comparison = run_real_seaman.compare_with_seaman(lambda_function=lambda_functions.K_h_function,
                                                        shipdict = shipdict,
                                                        df = df,label = 'mx')

fig,ax = plt.subplots()
result_comparison.plot(x = 'v_w',y = ['mx','mx_seaman'],ax = ax)
ax.set_title('Drift angle variation');

In [ ]:
df = pd.DataFrame()
df['r_w'] = np.linspace(-0.3,0.3,20)
df['rho'] = 1025.0
df['g'] = 9.81
df['u_w'] = 5.0
df['v_w'] = 0.0
df['p'] = 0

shipdict2 = shipdict.copy()
shipdict2.lin_hull_coeff_data['kur'] = 0
#shipdict2.lin_hull_coeff_data['kuur'] = 0

result_comparison = run_real_seaman.compare_with_seaman(lambda_function=lambda_functions.K_h_function,
                                                        shipdict = shipdict2,
                                                        df = df,label = 'mx')
fig,ax = plt.subplots()
result_comparison.plot(x = 'r_w',y = ['mx','mx_seaman'],ax = ax)
ax.set_title('Yaw rate variation');

In [ ]:
np.sqrt(df_input['L'])/2